In [ ]:
import json
from datasets import Dataset

# Load the training dataset (balanced version) from a JSON file.
# Each record is expected to contain the training text and classification label.
with open("train_balanced.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Output a single record for structural verification of the dataset.
print(train_data[1])

# Convert loaded records into a Hugging Face Dataset object
# This enables efficient preprocessing and model training compatibility.
train_dataset = Dataset.from_list(train_data)

# Display sample content and dataset schema to confirm correct field formatting.
print(train_dataset[0])
print(train_dataset.features)


{'text': 'Age:74.  Gender:female.  Diagnosis:rrms. Has not converted to SPMS. Initial presentation motor weakness, brainstem. Number of attacks until the first visit:2. OCBs in CSF:positive. Name of co-morbidity:Osteoporosis or osteopenia. More information on co-morbidity:Osteoporosis or osteopenia. Outcome:ongoing illness. Name of co-morbidity:Diabetes mellitus type 2. Outcome:ongoing illness. Name of co-morbidity:Gastrointestinal. More information on co-morbidity:Gastrointestinal. Outcome:completely recovered from illness. Name of co-morbidity:Upper respiratory tract infection. Outcome:completely recovered from illness. Name of co-morbidity:Surgeries. More information on co-morbidity:Surgeries. Outcome:ongoing illness. Name of co-morbidity:Musculoskeletal. More information on co-morbidity:hip replacement surgery. Outcome:completely recovered from illness. Visit diagnosis:rrms. Total relapses until visit: 3. DMT taken. Current DMT:mabthera- rituximab on April 09, 2013. Comments on DMT

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
# Import necessary libraries and load a large language model from Unsloth
from unsloth import FastLanguageModel
import torch

# Specify the pre-trained LLaMA model to fine-tune
model_name = "unsloth/Llama-3.2-3B-Instruct"

# Define the maximum sequence length for input prompts and outputs
max_seq_length = 2048
dtype = None  # Let Unsloth auto-detect the optimal data type

# Load the model and tokenizer, using 4-bit quantization to save GPU memory
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# Import the Hugging Face Dataset class to handle structured training data
from datasets import Dataset

# Define a function to format each example for supervised fine-tuning
def format_prompt(example):
    return f"### Text: {example['text']}\n### Output: {json.dumps(example['output'])}\n### Label: {example['label']}<|endoftext|>"

# Apply the formatting function to the entire training dataset
formatted_data = [format_prompt(item) for item in train_data]

# Convert the formatted data into a Hugging Face Dataset object for easy integration with the trainer
dataset = Dataset.from_dict({"text": formatted_data})


In [ ]:
from datasets import load_dataset

#ADD the validation dataset
val_dataset = load_dataset("json", data_files="val.json")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Integrate LoRA adapters into the base model to enable parameter-efficient fine-tuning
# This modifies only a subset of model weights, allowing faster training and lower memory usage
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank controlling adapter capacity (higher = more expressive, uses more memory)
    target_modules=[       # Specify which internal projection layers to adapt
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,        # Scaling factor for LoRA weights (commonly 2x rank)
    lora_dropout=0,        # Dropout applied to LoRA weights to improve generalization
    bias="none",           # Bias handling strategy optimized for Unsloth models
    use_gradient_checkpointing="unsloth",  # Enable memory-efficient gradient checkpointing
    random_state=3407,     # Fix random seed for reproducibility
    use_rslora=False,      # Disable rank-stabilized LoRA
    loftq_config=None,     # No LoftQ quantization config used
)


Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# Set up the supervised fine-tuning trainer for LoRA-adapted model
# Uses both training and validation datasets to monitor performance during training
trainer = SFTTrainer(
    model=model,                  # LoRA-adapted model
    tokenizer=tokenizer,          # Corresponding tokenizer
    train_dataset=dataset,        # Training dataset
    eval_dataset=val_dataset,     # Validation dataset for monitoring
    dataset_text_field="text",    # Field containing formatted prompt text
    max_seq_length=max_seq_length,  # Maximum sequence length for training
    dataset_num_proc=2,           # Number of parallel processes for dataset preprocessing
    args=TrainingArguments(
        per_device_train_batch_size=1,    # Number of samples per GPU
        gradient_accumulation_steps=1,    # Accumulate gradients to simulate larger batch
        warmup_steps=50,                  # Linear warmup steps for learning rate
        num_train_epochs=3,               # Number of training epochs
        learning_rate=5e-5,               # Base learning rate
        fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 not supported
        bf16=torch.cuda.is_bf16_supported(),      # Use BF16 if supported
        logging_steps=10,                  # Interval for logging training metrics
        optim="adamw_8bit",                # Optimizer with 8-bit precision for memory efficiency
        weight_decay=0.1,                  # L2 regularization
        lr_scheduler_type="linear",        # Linear learning rate decay
        seed=3407,                          # Random seed for reproducibility
        output_dir="outputs",              # Directory to save checkpoints and logs
        report_to="none",                  # Disable external logging (e.g., WandB)
        evaluation_strategy="epoch"        # Evaluate on validation set every epoch
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2076 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,076 | Num Epochs = 6 | Total steps = 12,456
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)


Step,Training Loss
10,3.181000
20,2.653400
30,1.961400
40,1.114600
50,0.622000
60,0.732400
70,0.426000
80,0.410600
90,0.380700
100,0.338500


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
import pandas as pd
import torch

# Switch the LoRA-adapted model to inference mode for evaluation
FastLanguageModel.for_inference(model)

# Load the test dataset
test_df = pd.read_json("test.json")

# Initialize lists to store classification results by category
TP = []  # True Positives
TN = []  # True Negatives
FP = []  # False Positives
FN = []  # False Negatives

# List to store all results in structured format
all_results = []

# Iterate over test examples and perform model inference
for idx, row in test_df.iterrows():
    text_input = row['text']              # Original input text
    true_output = row['output']           # Ground-truth structured output
    true_label = row['label']             # Ground-truth class label

    messages = [{"role": "user", "content": text_input}]

    # Tokenize input and prepare for model
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",  # Returns dictionary with input_ids and attention_mask
    ).to("cuda")

    # Generate model output using deterministic decoding (temperature=0, no sampling)
    outputs = model.generate(
        input_ids=inputs,
        attention_mask=torch.ones_like(inputs),
        max_new_tokens=256,
        use_cache=True,
        temperature=0.0,
        do_sample=False,
        top_p=0.9,
    )

    # Decode generated sequence, excluding input prompt
    generated_tokens = outputs[:, inputs.shape[1]:]
    pred_output = tokenizer.batch_decode(generated_tokens)[0].strip()

    # Robust label parsing from generated text
    pred_upper = pred_output.upper()
    if "NEDA" in pred_upper:
        pred_label = "NEDA"
    elif "EDA" in pred_upper:
        pred_label = "EDA"
    else:
        pred_label = "NEDA"  # Default fallback
        print(f"⚠️ WARNING: Unexpected model output: '{pred_output}'")

    # Classify prediction into TP, TN, FP, FN
    match = ""
    if true_label == "EDA":
        if pred_label == "EDA":
            TP.append([text_input, true_output, true_label, pred_output, pred_label, "TP"])
            match = "TP"
        else:
            FN.append([text_input, true_output, true_label, pred_output, pred_label, "FN"])
            match = "FN"
    else:
        if pred_label == "EDA":
            FP.append([text_input, true_output, true_label, pred_output, pred_label, "FP"])
            match = "FP"
        else:
            TN.append([text_input, true_output, true_label, pred_output, pred_label, "TN"])
            match = "TN"

    all_results.append([text_input, true_output, true_label, pred_output, pred_label, match])

# Compute evaluation metrics
num_TP = len(TP)
num_TN = len(TN)
num_FP = len(FP)
num_FN = len(FN)

total_samples = num_TP + num_TN + num_FP + num_FN
accuracy = (num_TP + num_TN) / total_samples if total_samples > 0 else 0
precision = num_TP / (num_TP + num_FP) if (num_TP + num_FP) > 0 else 0
recall = num_TP / (num_TP + num_FN) if (num_TP + num_FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Display metrics and counts
print("Metrics:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

print("\nCounts:")
print(f"TP: {num_TP}, TN: {num_TN}, FP: {num_FP}, FN: {num_FN}")


Metrics:
Accuracy: 0.569
Precision: 0.857
Recall: 0.113
F1 Score: 0.200

Counts:
TP: 24, TN: 229, FP: 4, FN: 188


In [ ]:
# Convert all evaluation results into a structured DataFrame for saving and analysis
results_df = pd.DataFrame(
    all_results,
    columns=[
        "text",          # Original input text
        "true_output",   # Ground-truth structured output
        "true_label",    # Ground-truth class label (EDA / NEDA)
        "pred_output",   # Raw text generated by the model
        "pred_label",    # Parsed predicted class label (EDA / NEDA)
        "result_type"    # Classification outcome category: TP, TN, FP, FN
    ],
)

# Inspect the first few rows to verify format before exporting
results_df.head()


In [ ]:
results_df.to_excel("model_predictions_comparison.xlsx", index=False)

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

# Locate the fine-tuned GGUF model file and download it to the local machine for use or sharing
gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)
